## Loading data

In [1]:
import pandas as pd

In [2]:
import nltk
import spacy
from nltk.util import ngrams
import natasha

In [3]:
df_full = pd.read_csv("../data/df_5_percent_sample.csv") # 5% of the whole dataset = frac 0.05, random_state 42

In [4]:
df_full.dropna(subset=['message'], inplace=True)

df = df_full.sample(frac=0.1, random_state=42).copy()

In [6]:
len(df) #0.5% from total

35068

## Testing effectiveness and processing speed for spacy and natasha for NER extraction

## Natasha

In [30]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsNERTagger,
    NewsMorphTagger,
    NewsSyntaxParser,
    PER,
    NamesExtractor,
    Doc
)

In [32]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [33]:
text = "Москве - столица России и одна из крупнейших помоек в мире."
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

In [61]:
for span in doc.spans:
    span.normalize(morph_vocab)
    print(f"{span.normal} [{span.type}, {doc.sents[0].morph}]")

Москва [LOC, MorphMarkup(tokens=[MorphToken(text='Москве', pos='PROPN', feats={'Animacy': 'Inan', 'Case': 'Loc', 'Gender': 'Fem', 'Number': 'Sing'}), MorphToken(text='-', pos='PUNCT', feats={}), MorphToken(text='столица', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), MorphToken(text='России', pos='PROPN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Fem', 'Number': 'Sing'}), MorphToken(text='и', pos='CCONJ', feats={}), MorphToken(text='одна', pos='NUM', feats={'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), MorphToken(text='из', pos='ADP', feats={}), MorphToken(text='крупнейших', pos='ADJ', feats={'Case': 'Gen', 'Degree': 'Sup', 'Number': 'Plur'}), MorphToken(text='помоек', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Fem', 'Number': 'Plur'}), MorphToken(text='в', pos='ADP', feats={}), MorphToken(text='мире', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Loc', 'Gender': 'Masc', 'Number': 'Sing'}), MorphToken(tex

In [56]:
def extract_NER_natasha(text):
    if not isinstance(text, str):
        return []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    return [(span.normal, span.type) for span in doc.spans]

In [55]:
def apply_ner_to_dataframe(df, column_name):
    return df[column_name].apply(extract_NER_natasha)

In [38]:
import time

start_time = time.time()

df['NER_Natasha_norm'] = apply_ner_to_dataframe(df, 'message')

print(f"Execution time: {time.time() - start_time:.4f} seconds")

Execution time: 419.5577 seconds


In [40]:
df_10 = df.sample(10).copy()

print(df_10.iloc[0].message)

print(df_10.iloc[0].NER_Natasha_norm)

Лежат Сталин и Ленин в ванной и вдруг бац, сперма всплывает.
- Иосиф, это ты кончил или пернул?
- Незнаю, Вова. Историки до сих пор во мнениях не сходятся..
[('Ленин', 'PER'), ('Иосиф', 'PER'), ('Незнаю', 'PER'), ('Вова', 'PER')]


In [51]:
df_10 = df.sample(10).copy()

print(df_10.iloc[0].message)

print(df_10.iloc[0].NER_Natasha_norm)

Да, вчера много писали, что по законам современной войны - медиа это второй, а возможно и первый фронт. И начать операцию на Украине в ответ за то, что западные спортивные чиновники обидели нашу золотую девочку Валиеву - это  самое то, что нужно. 

Причём, дипломаты должны будут так и объяснять - да при чем тут НАТО? Вы Валиеву унизили. За это мы размотаем ваше предполье.

И вот теперь оказывается, что Валиева выступить сможет. Но по законам современной войны и медиавойны, по законам мятежвойны, это все не повод отменить вторжение. "Ты извинился, но сделал это без уважения". 

Ну или пусть не вторжение, и без него модно создать напряжение. Второй информационный фронт  всемирной мятежвойны. Самое время завирусить в интернете (а лучше в Телеграме у русских военкоров) видео, где русские лётчики пишут на бомбах "За Камиллу Валиеву".

А на ракетах артиллеристы пишут "За жирафа Мариуса".

Репутация злопамятных и непрощающих парней без тормозов - это ценно. Тем более, что они на Западе начита

In [52]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
word = "Валиеву"

parsed_word = morph.parse(word)[0]

plural_form = parsed_word.inflect({'plur'})
print(f"Plural form: {plural_form.word}")

genitive_form = parsed_word.inflect({'gent'})
print(f"Genitive form: {genitive_form.word}")

dative_form = parsed_word.inflect({'datv'})
print(f"Dative form: {dative_form.word}")

Plural form: валиевым
Genitive form: валиева
Dative form: валиеву


In [53]:
parsed_word = morph.parse(word)[0]
all_forms = parsed_word.lexeme
for form in all_forms:
    print(form.word, form.tag)

валиев NOUN,anim,masc,Sgtm,Surn sing,nomn
валиева NOUN,anim,masc,Sgtm,Surn sing,gent
валиеву NOUN,anim,masc,Sgtm,Surn sing,datv
валиева NOUN,anim,masc,Sgtm,Surn sing,accs
валиевым NOUN,anim,masc,Sgtm,Surn sing,ablt
валиеве NOUN,anim,masc,Sgtm,Surn sing,loct
валиева NOUN,anim,femn,Sgtm,Surn sing,nomn
валиевой NOUN,anim,femn,Sgtm,Surn sing,gent
валиевой NOUN,anim,femn,Sgtm,Surn sing,datv
валиеву NOUN,anim,femn,Sgtm,Surn sing,accs
валиевой NOUN,anim,femn,Sgtm,Surn sing,ablt
валиевой NOUN,anim,femn,Sgtm,Surn sing,loct
валиевы NOUN,anim,ms-f,Pltm,Surn plur,nomn
валиевых NOUN,anim,ms-f,Pltm,Surn plur,gent
валиевым NOUN,anim,ms-f,Pltm,Surn plur,datv
валиевых NOUN,anim,ms-f,Pltm,Surn plur,accs
валиевыми NOUN,anim,ms-f,Pltm,Surn plur,ablt
валиевых NOUN,anim,ms-f,Pltm,Surn plur,loct


In [65]:
df_10 = df.sample(10).copy()

print(df_10.iloc[0].message)

print(df_10.iloc[0].NER_Natasha_norm)

🇷🇺🇺🇦 Брифинг Минобороны России (17.04.2022 г.)

◽️ Вооруженные Силы Российской Федерации продолжают специальную военную операцию на Украине.
 
💥 В течение дня высокоточными ракетами воздушного базирования в районе населенных пунктов БАРВЕНКОВО и ДОБРОПОЛЬЕ уничтожены хранилища горюче-смазочных материалов и боеприпасов.
 
💥 Оперативно-тактической авиацией уничтожены 44 военных объектов Украины. Среди них: в районе АВДЕЕВКА два командных пункта и локатор подсвета и наведения целей зенитного ракетного комплекса С-300; в районах ЗАВГОРОДНЕЕ и ПРОТОПОПОВКА уничтожено три склада ракетно-артиллерийского вооружения; в районах КРАСНЫЙ ЛИМАН, НОВОСЕЛОВКА, РУБЕЖНОЕ, УГЛЕДАР, ПОПАСНАЯ, ПРИШИБ и ГУСАРОВКА 23 места сосредоточения личного состава и украинской военной техники. 
 
💥 Ракетными войсками и артиллерией нанесено поражение 113-ти объектам, в том числе: четырем пунктам управления, четырем артиллерийским батареям, двум складам горюче-смазочных материалов и 103-м опорным пунктам и районам сосре

In [66]:
df_10 = df.sample(10).copy()

print(df_10.iloc[0].message)

print(df_10.iloc[0].NER_Natasha_norm)

Одновременно с местными выборами на Украине пройдет общенациональный опрос. На него вынесены 5 вопросов:

▪️ нужно ли пожизненное заключение за крупную коррупцию

▪️ создать ли в Донбассе свободную экономическую зону

▪️ сократить ли Верховную раду с 450 до 300 депутатов

▪️ легализовать ли медицинскую марихуану

▪️ следует ли пересмотреть Будапештский меморандум, согласно которому Киев отказался от ядерного оружия в обмен на гарантии своей целостности

Опрос - в отличие от референдума - консультативный и не будут иметь юридической силы.

Оппоненты президента считают это попыткой повысить явку сторонников его партии и повысить ее результат на выборах. Такого же мнения 44% украинцев - еще 44% считают опрос искренней попыткой узнать мнение избирателей (исследование КМИС).

Участвовать во всеукраинском опросе сегодня намерены 55% респондентов, не собираются 36%.
[('Украина', 'LOC'), ('Донбасс', 'LOC'), ('Верховная рада', 'ORG'), ('Киев', 'LOC'), ('КМИС', 'ORG')]


In [68]:
df_10 = df.sample(10).copy()

print(df_10.iloc[0].message)

print(df_10.iloc[0].NER_Natasha_norm)

Уххх, и жарким выдался вечерок на Тавриде. Если вы ещё не понимаете о чем мы, то мы вам расскажем. 8 сентября в Крыму открылся молодежный фестиваль для талантливых людей «Таврида. АРТ».

На площадке фестиваля под открытым небом встретилось более 5 тыс. участников. И все они собрались для того, чтобы творить. Музыканты, артисты, писатели, художники — все они лица современной Российской культуры. 

Но и скучать не придётся: в программе обучающие мастер-классы, творческие конкурсы, лекции от победителей конкурсов платформы «Россия — страна возможностей». Держим пари, вы тоже туда захотели?
[('Уххх', 'LOC'), ('Крым', 'LOC'), ('Российская культура', 'LOC')]


1. Зброю він не знаходить як NER
2. Місспеллінг або еративи вважає Нерами

### Spacy

In [19]:
nlp = spacy.load('ru_core_news_sm', disable=["tagger", "parser"])

In [20]:
text = "Москва - столица России и одна из крупнейших помоек в мире."
doc = nlp(text)

In [21]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Москва - столица 0 16 LOC
России 17 23 LOC


In [22]:
def extract_named_entities_spacy(text):
    if not isinstance(text, str):
        return []

    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [23]:
def apply_ner_to_dataframe_spacy(df, column_name):
    return df[column_name].apply(extract_named_entities_spacy)

In [24]:
# Measure execution time
start_time = time.time()

# Apply NER
df['NER_Spacy'] = apply_ner_to_dataframe_spacy(df, 'message')

# Print execution time
print(f"Execution time: {time.time() - start_time:.4f} seconds")

Execution time: 674.8227 seconds


In [25]:
df.sample(20)

Unnamed: 0.1  Unnamed: 0        id                       date  \
50983        2292047       38433   63317.0  2021-08-13 10:18:28+00:00   
382743       7822300      294657  144430.0  2019-05-13 07:12:18+00:00   
155945       5193712       43704   57759.0  2021-12-30 14:50:09+00:00   
201495       6977027       32669    9918.0  2019-12-29 14:13:49+00:00   
60554        2972354        6999    9525.0  2020-05-09 08:56:33+00:00   
316373       3689082        2698   12039.0  2022-03-19 13:42:19+00:00   
383953       5224048       74040   26894.0  2020-11-02 10:39:17+00:00   
370080       2904775       11412    5413.0  2020-10-19 09:05:26+00:00   
323916       5485778       74660    1221.0  2018-07-28 05:41:08+00:00   
260134       6571338         891     506.0  2022-05-17 09:04:21+00:00   
321906       4424387       25244  119576.0  2022-08-05 04:09:02+00:00   
166016       2341389       87775   12929.0  2018-12-03 21:17:44+00:00   
268288       4309676       25887   80188.0  2021-05-12 11:16:51+00:00   
57109        6161111        8870   20348.0  2022-08-18 19:18:28+00:00   
95287        6614724       10202   11560.0  2020-09-16 12:15:17+00:00   
210202        688256        2835    7442.0  2022-02-25 15:37:38+00:00   
267796       5894347        2931     167.0  2018-03-16 16:04:07+00:00   
385461       2728467       11708   32612.0  2022-10-17 07:57:37+00:00   
25149        7569215       41572  404641.0  2022-07-23 05:04:08+00:00   
10974        6976619       32261   10335.0  2020-01-13 22:47:16+00:00   

           views                                          reactions  \
50983     8322.0                                                NaN   
382743       1.0                                                NaN   
155945    7582.0  MessageReactions(results=[ReactionCount(reacti...   
201495       6.0                                                NaN   
60554    30058.0                                                NaN   
316373  111372.0  MessageReactions(results=[ReactionCount(reacti...   
383953    5614.0                                                NaN   
370080    4405.0                                                NaN   
323916       1.0                                                NaN   
260134  221375.0  MessageReactions(results=[ReactionCount(reacti...   
321906  412188.0  MessageReactions(results=[ReactionCount(reacti...   
166016   13384.0                                                NaN   
268288   10585.0                                                NaN   
57109     3288.0  MessageReactions(results=[ReactionCount(reacti...   
95287      917.0                                                NaN   
210202   37455.0                                                NaN   
267796   45313.0  MessageReactions(results=[ReactionCount(reacti...   
385461  145360.0  MessageReactions(results=[ReactionCount(reacti...   
25149    15984.0                                                NaN   
10974        2.0                                                NaN   

                                     to_id  \
50983   PeerChannel(channel_id=1046446760)   
382743  PeerChannel(channel_id=1082084045)   
155945  PeerChannel(channel_id=1111072403)   
201495  PeerChannel(channel_id=1283359437)   
60554   PeerChannel(channel_id=1109145894)   
316373  PeerChannel(channel_id=1004504016)   
383953  PeerChannel(channel_id=1111072403)   
370080  PeerChannel(channel_id=1120807475)   
323916  PeerChannel(channel_id=1216878621)   
260134  PeerChannel(channel_id=1595839251)   
321906  PeerChannel(channel_id=1315735637)   
166016  PeerChannel(channel_id=1046446760)   
268288  PeerChannel(channel_id=1054549314)   
57109   PeerChannel(channel_id=1138542535)   
95287   PeerChannel(channel_id=1134554551)   
210202  PeerChannel(channel_id=1042737298)   
267796  PeerChannel(channel_id=1141171940)   
385461  PeerChannel(channel_id=1453804949)   
25149   PeerChannel(channel_id=1082084045)   
10974   PeerChannel(channel_id=1283359437)   

               

## NER Normalization

In [31]:
import razdel
import navec
import slovnet
import pandas

In [30]:
# def download_file(url, filename):
#     if not os.path.exists(filename):
#         urllib.request.urlretrieve(url, filename)
# 
# download_file("https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar", "navec_hudlit_v1_12B_500K_300d_100q.tar")
# download_file("https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_bert.tar", "slovnet_ner_bert.tar")

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>